# Chapter 2: Backtesting

## This Chapter
We backtest our Core ETF (MSCI ACWI) to establish a baseline performance:
1. Load price data from our SQLite database
2. Run backtest with the Backtester module
3. Calculate performance metrics (Sharpe, Sortino, Max Drawdown, etc.)
4. Visualize portfolio growth and drawdowns

## Output
- Baseline performance metrics for 100% Core portfolio
- This serves as the benchmark for satellite strategies in later chapters

## 2.1 Setup & Imports

In [ ]:
# Imports
import sys
from pathlib import Path

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Add current directory to path
sys.path.insert(0, str(Path.cwd()))

# Configure Plotly template with transparent background
pio.templates["custom_dark"] = pio.templates["plotly_dark"]
pio.templates["custom_dark"].layout.paper_bgcolor = 'rgba(0,0,0,0)'
pio.templates["custom_dark"].layout.plot_bgcolor = 'rgba(0,0,0,0)'
pio.templates.default = "custom_dark"

print("Imports loaded")

In [ ]:
# Color palette
COLORS = {
    'primary': '#2196F3',    # Blue
    'secondary': '#9E9E9E',  # Gray
    'profit': '#4CAF50',     # Green
    'loss': '#F44336',       # Red
    'accent': '#9C27B0',     # Purple
    'warning': '#FF9800',    # Orange
    'highlight': '#E91E63',  # Pink
}

print("Colors configured")

## 2.2 Load Data from Database

In [3]:
from etf_database import ETFDatabase

# Connect to database
db = ETFDatabase("data/etf_database.db")

stats = db.get_stats()
print(f"Database: {stats['total_etfs']} ETFs, {stats['total_price_records']:,} price records")
print(f"Date range: {stats['price_date_range'][0]} to {stats['price_date_range'][1]}")

Database: 872 ETFs, 2,063,930 price records
Date range: 2005-11-18 to 2025-12-12


In [4]:
# Define our Core ETF
CORE_ISIN = 'IE00B6R52259'  # iShares MSCI ACWI UCITS ETF USD (Acc)

# Get Core ETF info
core_info = db.get_etf(CORE_ISIN)
if core_info:
    print(f"Core ETF: {CORE_ISIN}")
    print(f"  Name: {core_info['name']}")
    print(f"  TER: {core_info['ter']:.2f}%")
    print(f"  Fund Size: {core_info['fund_size']:,.0f}M EUR")
    print(f"  Months of Data: {core_info['months_of_data']}")
else:
    print(f"ERROR: Core ETF {CORE_ISIN} not found in database!")

Core ETF: IE00B6R52259
  Name: iShares MSCI ACWI UCITS ETF USD (Acc) (EUR)
  TER: 0.20%
  Fund Size: 20,924M EUR
  Months of Data: 169


In [5]:
# Load Core ETF prices
core_prices = db.load_prices(CORE_ISIN)

print(f"Price data loaded:")
print(f"  Date range: {core_prices.index.min().date()} to {core_prices.index.max().date()}")
print(f"  Data points: {len(core_prices):,}")
print(f"  Latest price: {core_prices.iloc[-1]:.2f} EUR")

# Show sample
print(f"\nRecent prices:")
print(core_prices.tail())

Price data loaded:
  Date range: 2011-10-21 to 2025-12-12
  Data points: 5,167
  Latest price: 92.00 EUR

Recent prices:
date
2025-12-08    92.51
2025-12-09    92.51
2025-12-10    93.02
2025-12-11    92.72
2025-12-12    92.00
Name: IE00B6R52259, dtype: float64


## 2.3 Backtest Core ETF

In [6]:
from backtester import Backtester, PortfolioMetrics, BacktestResult

# Convert Series to DataFrame for backtester
prices_df = core_prices.to_frame(name=CORE_ISIN)

# Create backtester for 100% Core portfolio
bt = Backtester(
    prices=prices_df,
    core_isin=CORE_ISIN,
    satellite_isins={},  # No satellites - 100% Core
    core_weight=1.0,
)

print("Backtester initialized for 100% Core portfolio")

Backtester initialized for 100% Core portfolio


In [21]:
# Backtest configuration
START_DATE = '2020-03-21'  # Set to '2020-01-01' to start from specific date, or None for full history
INITIAL_INVESTMENT = 50000  # EUR
MONTHLY_CONTRIBUTION = 1000  # EUR

# Run backtest
result = bt.run(
    start_date=START_DATE,
    initial_investment=INITIAL_INVESTMENT,
    monthly_contribution=MONTHLY_CONTRIBUTION,
    verbose=False
)

print(f"Backtest completed")
print(f"  Period: {result.portfolio.index[0].date()} to {result.portfolio.index[-1].date()}")
print(f"  Trading days: {len(result.portfolio):,}")
print(f"  Rebalances: {len(result.rebalance_dates)}")

Backtest completed
  Period: 2020-03-21 to 2025-12-12
  Trading days: 2,093
  Rebalances: 70


In [22]:
# Calculate performance metrics
metrics = PortfolioMetrics.calculate(result.portfolio)

# Print summary
print("="*60)
print("CORE ETF PERFORMANCE (100% MSCI ACWI)")
print("="*60)
PortfolioMetrics.print_summary(metrics)

# Additional stats
years = (result.portfolio.index[-1] - result.portfolio.index[0]).days / 365.25
total_invested = INITIAL_INVESTMENT + (MONTHLY_CONTRIBUTION * len(result.rebalance_dates))
final_value = result.portfolio.iloc[-1]
profit = final_value - total_invested

print(f"\nInvestment Summary:")
print(f"  Period: {years:.1f} years")
print(f"  Initial: {INITIAL_INVESTMENT:,.0f} EUR")
print(f"  Monthly: {MONTHLY_CONTRIBUTION:,.0f} EUR")
print(f"  Total Invested: {total_invested:,.0f} EUR")
print(f"  Final Value: {final_value:,.0f} EUR")
print(f"  Profit: {profit:,.0f} EUR ({profit/total_invested*100:.1f}%)")
print("="*60)

CORE ETF PERFORMANCE (100% MSCI ACWI)
Portfolio Performance Metrics
  Total Return:        353.91%
  Annualized Return:    19.99%
  Annualized Vol:       13.05%
  Sharpe Ratio:         1.532
  Sortino Ratio:        1.866
  Calmar Ratio:         1.045
  Max Drawdown:        -19.12%

Investment Summary:
  Period: 5.7 years
  Initial: 50,000 EUR
  Monthly: 1,000 EUR
  Total Invested: 120,000 EUR
  Final Value: 226,956 EUR
  Profit: 106,956 EUR (89.1%)


## 2.4 Visualize Performance

In [ ]:
# Calculate invested amount over time
invested = pd.Series(index=result.portfolio.index, dtype=float)
cumulative = INITIAL_INVESTMENT
last_month = None
for date in invested.index:
    if last_month is None or date.month != last_month:
        if last_month is not None:
            cumulative += MONTHLY_CONTRIBUTION
        last_month = date.month
    invested[date] = cumulative

# Calculate drawdown
rolling_max = result.portfolio.cummax()
drawdown = (result.portfolio - rolling_max) / rolling_max * 100

# Create subplots: Portfolio Value and Drawdown
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.08,
    row_heights=[0.7, 0.3],
    subplot_titles=(
        f'Portfolio Growth: {core_info["name"]}',
        'Drawdown'
    )
)

# Portfolio value
fig.add_trace(
    go.Scatter(
        x=result.portfolio.index,
        y=result.portfolio.values,
        name='Portfolio Value',
        line=dict(color=COLORS['primary'], width=2),
        hovertemplate='Date: %{x|%Y-%m-%d}<br>Value: €%{y:,.0f}<extra></extra>'
    ),
    row=1, col=1
)

# Invested amount
fig.add_trace(
    go.Scatter(
        x=invested.index,
        y=invested.values,
        name='Total Invested',
        line=dict(color=COLORS['secondary'], width=1.5, dash='dash'),
        hovertemplate='Date: %{x|%Y-%m-%d}<br>Invested: €%{y:,.0f}<extra></extra>'
    ),
    row=1, col=1
)

# Profit area (fill between)
fig.add_trace(
    go.Scatter(
        x=result.portfolio.index,
        y=result.portfolio.values,
        fill='tonexty',
        fillcolor='rgba(76, 175, 80, 0.2)',
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    row=1, col=1
)

# Drawdown
fig.add_trace(
    go.Scatter(
        x=drawdown.index,
        y=drawdown.values,
        name='Drawdown',
        fill='tozeroy',
        fillcolor='rgba(244, 67, 54, 0.3)',
        line=dict(color=COLORS['loss'], width=1),
        hovertemplate='Date: %{x|%Y-%m-%d}<br>Drawdown: %{y:.1f}%<extra></extra>'
    ),
    row=2, col=1
)

# Max drawdown line
fig.add_hline(
    y=metrics['max_drawdown']*100,
    line_dash='dash',
    line_color=COLORS['highlight'],
    annotation_text=f"Max DD: {metrics['max_drawdown']*100:.1f}%",
    annotation_position='bottom right',
    row=2, col=1
)

# Update layout
fig.update_layout(
    height=700,
    legend=dict(x=0.01, y=0.99, bgcolor='rgba(0,0,0,0)'),
)

fig.update_yaxes(title_text='Value (EUR)', tickformat=',.0f', row=1, col=1)
fig.update_yaxes(title_text='Drawdown (%)', row=2, col=1)
fig.update_xaxes(title_text='Date', row=2, col=1)

fig.show()

In [ ]:
# Calculate rolling metrics (252 trading days = ~1 year)
returns = result.portfolio.pct_change().dropna()
rolling_window = 252

# Rolling returns (annualized)
rolling_return = (1 + returns).rolling(rolling_window).apply(
    lambda x: (x.prod() ** (252/len(x))) - 1, raw=False
) * 100

# Rolling volatility (annualized)
rolling_vol = returns.rolling(rolling_window).std() * np.sqrt(252) * 100

# Rolling Sharpe
rolling_sharpe = rolling_return / rolling_vol

# Create 2x2 subplot
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Rolling 1-Year Annualized Return',
        'Rolling 1-Year Volatility',
        'Rolling 1-Year Sharpe Ratio',
        'Monthly Returns Heatmap'
    ),
    vertical_spacing=0.12,
    horizontal_spacing=0.08
)

# Rolling return
fig.add_trace(
    go.Scatter(
        x=rolling_return.index,
        y=rolling_return.values,
        name='Rolling Return',
        line=dict(color=COLORS['primary'], width=1.5),
        hovertemplate='Date: %{x|%Y-%m-%d}<br>Return: %{y:.1f}%<extra></extra>'
    ),
    row=1, col=1
)
fig.add_hline(y=0, line_dash='dot', line_color='white', opacity=0.3, row=1, col=1)
fig.add_hline(y=metrics['ann_return']*100, line_dash='dash', line_color=COLORS['profit'],
              annotation_text=f"Overall: {metrics['ann_return']*100:.1f}%", row=1, col=1)

# Rolling volatility
fig.add_trace(
    go.Scatter(
        x=rolling_vol.index,
        y=rolling_vol.values,
        name='Rolling Volatility',
        line=dict(color=COLORS['warning'], width=1.5),
        hovertemplate='Date: %{x|%Y-%m-%d}<br>Volatility: %{y:.1f}%<extra></extra>'
    ),
    row=1, col=2
)
fig.add_hline(y=metrics['ann_vol']*100, line_dash='dash', line_color=COLORS['profit'],
              annotation_text=f"Overall: {metrics['ann_vol']*100:.1f}%", row=1, col=2)

# Rolling Sharpe
fig.add_trace(
    go.Scatter(
        x=rolling_sharpe.index,
        y=rolling_sharpe.values,
        name='Rolling Sharpe',
        line=dict(color=COLORS['accent'], width=1.5),
        hovertemplate='Date: %{x|%Y-%m-%d}<br>Sharpe: %{y:.2f}<extra></extra>'
    ),
    row=2, col=1
)
fig.add_hline(y=0, line_dash='dot', line_color='white', opacity=0.3, row=2, col=1)
fig.add_hline(y=metrics['sharpe'], line_dash='dash', line_color=COLORS['profit'],
              annotation_text=f"Overall: {metrics['sharpe']:.2f}", row=2, col=1)

# Monthly returns heatmap
monthly_returns = result.portfolio.resample('ME').last().pct_change().dropna() * 100
monthly_df = pd.DataFrame({
    'Year': monthly_returns.index.year,
    'Month': monthly_returns.index.month,
    'Return': monthly_returns.values
})
pivot = monthly_df.pivot(index='Year', columns='Month', values='Return')

fig.add_trace(
    go.Heatmap(
        z=pivot.values,
        x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        y=pivot.index,
        colorscale='RdYlGn',
        zmid=0,
        zmin=-10,
        zmax=10,
        colorbar=dict(title='Return %', x=1.02),
        hovertemplate='%{y} %{x}<br>Return: %{z:.1f}%<extra></extra>'
    ),
    row=2, col=2
)

# Update layout
fig.update_layout(
    height=700,
    showlegend=False,
)

fig.update_yaxes(title_text='Return (%)', row=1, col=1)
fig.update_yaxes(title_text='Volatility (%)', row=1, col=2)
fig.update_yaxes(title_text='Sharpe Ratio', row=2, col=1)
fig.update_xaxes(title_text='Date', row=2, col=1)

fig.show()

## 2.5 Backtest Without Contributions

For cleaner performance comparison, also run a backtest without monthly contributions.

In [25]:
# Run backtest without contributions (pure performance)
result_no_contrib = bt.run(
    start_date=START_DATE,
    initial_investment=10000,  # Normalized starting point
    monthly_contribution=0,
    verbose=False
)

metrics_no_contrib = PortfolioMetrics.calculate(result_no_contrib.portfolio)

print("="*60)
print("PURE PERFORMANCE (No Contributions)")
print("="*60)
PortfolioMetrics.print_summary(metrics_no_contrib)

# Calculate CAGR
years = (result_no_contrib.portfolio.index[-1] - result_no_contrib.portfolio.index[0]).days / 365.25
cagr = (result_no_contrib.portfolio.iloc[-1] / result_no_contrib.portfolio.iloc[0]) ** (1/years) - 1
print(f"\nCAGR: {cagr*100:.2f}%")
print(f"Period: {years:.1f} years")
print("="*60)

PURE PERFORMANCE (No Contributions)
Portfolio Performance Metrics
  Total Return:        154.35%
  Annualized Return:    11.90%
  Annualized Vol:       12.79%
  Sharpe Ratio:         0.931
  Sortino Ratio:        1.094
  Calmar Ratio:         0.597
  Max Drawdown:        -19.93%

CAGR: 17.70%
Period: 5.7 years


In [ ]:
# Normalize to 100 for comparison
normalized = result_no_contrib.portfolio / result_no_contrib.portfolio.iloc[0] * 100

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=normalized.index,
        y=normalized.values,
        name='MSCI ACWI',
        line=dict(color=COLORS['primary'], width=2),
        fill='tozeroy',
        fillcolor='rgba(33, 150, 243, 0.1)',
        hovertemplate='Date: %{x|%Y-%m-%d}<br>Value: %{y:.1f}<extra></extra>'
    )
)

# Starting line
fig.add_hline(y=100, line_dash='dot', line_color='white', opacity=0.5)

fig.update_layout(
    title=dict(
        text=f'Core ETF Performance (Normalized to 100) - CAGR: {cagr*100:.1f}%',
        font=dict(size=14)
    ),
    height=500,
    yaxis_title='Value (Base 100)',
    xaxis_title='Date'
)

fig.show()

## 2.6 Summary

In [27]:
# Final summary
print("="*70)
print("CHAPTER 2 SUMMARY: CORE ETF BASELINE")
print("="*70)
print(f"\nCore ETF: {CORE_ISIN}")
print(f"  {core_info['name']}")
print(f"  TER: {core_info['ter']:.2f}%")
print()
print(f"Backtest Period: {result.portfolio.index[0].date()} to {result.portfolio.index[-1].date()}")
print(f"  Duration: {years:.1f} years")
print()
print("Performance Metrics (Pure - No Contributions):")
print(f"  CAGR:           {cagr*100:>7.2f}%")
print(f"  Total Return:   {metrics_no_contrib['total_return']*100:>7.1f}%")
print(f"  Ann. Volatility:{metrics_no_contrib['ann_vol']*100:>7.1f}%")
print(f"  Sharpe Ratio:   {metrics_no_contrib['sharpe']:>7.2f}")
print(f"  Sortino Ratio:  {metrics_no_contrib['sortino']:>7.2f}")
print(f"  Calmar Ratio:   {metrics_no_contrib['calmar']:>7.2f}")
print(f"  Max Drawdown:   {metrics_no_contrib['max_drawdown']*100:>7.1f}%")
print()
print("This serves as the BENCHMARK for satellite strategies.")
print("Goal: Find satellite allocations that improve risk-adjusted returns.")
print("="*70)

CHAPTER 2 SUMMARY: CORE ETF BASELINE

Core ETF: IE00B6R52259
  iShares MSCI ACWI UCITS ETF USD (Acc) (EUR)
  TER: 0.20%

Backtest Period: 2020-03-21 to 2025-12-12
  Duration: 5.7 years

Performance Metrics (Pure - No Contributions):
  CAGR:             17.70%
  Total Return:     154.4%
  Ann. Volatility:   12.8%
  Sharpe Ratio:      0.93
  Sortino Ratio:     1.09
  Calmar Ratio:      0.60
  Max Drawdown:     -19.9%

This serves as the BENCHMARK for satellite strategies.
Goal: Find satellite allocations that improve risk-adjusted returns.


---

**Chapter 2 complete.** We have established the baseline performance of our Core ETF (MSCI ACWI).

**Next steps:**
- Chapter 3: Add satellite ETFs and test different allocation strategies
- Compare Core+Satellite portfolios against this baseline